In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 2000)

In [51]:
files=['Report_Card_Teacher_Qualification_Summary.csv',
'Report_Card_Assessment_Data_2018-19_School_Year.csv',
'Report_Card_Enrollment_2018-19_School_Year.csv',
'Report_Card_Teacher_Demographics_2018-19_School_Year.csv']

df1=pd.read_csv(files[0], low_memory=False)
df2=pd.read_csv(files[1], low_memory=False)
df3=pd.read_csv(files[2], low_memory=False)
df4=pd.read_csv(files[3], low_memory=False)
df5=pd.read_excel('Report Card Per Pupil Finance Data 2018-19.xlsx')

In [52]:
df5.DistrictName=df5.DistrictName.str.replace('Seattle School District #1','Seattle School District No. 1')\
                .str.replace('Green Dot Destiny ','Green Dot Destiny')

In [53]:
districts=df5.DistrictName.value_counts().sort_index()


In [54]:
master=[]
outs4=[]
for district in list(df4.DistrictName.value_counts().sort_index().index):
    if district not in districts:
        outs4.append(district)
    if district in districts:
        master.append(district)

In [55]:
outs3=[]
for district in list(df3.DistrictName.value_counts().sort_index().index):
    if district not in districts:
        outs3.append(district)
    if district in districts:
        master.append(district)

In [56]:
outs2=[]
for district in list(df3.DistrictName.value_counts().sort_index().index):
    if district not in districts:
        outs2.append(district)
    if district in districts:
        master.append(district)

In [57]:
outs1=[]
for district in list(df3.DistrictName.value_counts().sort_index().index):
    if district not in districts:
        outs1.append(district)
    if district in districts:
        master.append(district)

In [58]:
master=pd.Series(master).value_counts()

In [59]:
[i for i in list(districts.index) if i not in master.index]

['Impact Public Schools']

## Test Scores

In [60]:
sbac=\
df2.query("GradeLevel=='10th Grade' and\
           TestAdministration=='SBAC' and\
           TestSubject=='Math' and\
           StudentGroup in ('All Students',\
                            'Asian','Black/ African American',\
                            'Hispanic/ Latino of any race(s)','White') and\
           SchoolName=='District Total'")


In [61]:
notnull_sbac=sbac.dropna(subset=['CountMetStandard']).copy()

In [62]:
notnull_sbac.PercentMetStandard=notnull_sbac.PercentMetStandard.str.strip('%').astype(float)/100

In [63]:
percent_met=\
notnull_sbac[['DistrictName','StudentGroup','PercentMetStandard']]\
.pivot(index='DistrictName', columns='StudentGroup').droplevel(0,axis=1)

In [64]:
percent_met['students_testing_n']=\
sbac[sbac.StudentGroup=='All Students'].groupby('DistrictName')\
                                       .mean()['Count of Students Expected to Test']

In [65]:
percent_met.columns=['sbac_pass_all',
                     'sbac_pass_asian',
                     'sbac_pass_black',
                     'sbac_pass_latinx',
                     'sbac_pass_white',
                     'students_testing_n']

In [66]:
percent_met=\
percent_met[['students_testing_n',
             'sbac_pass_all',
             'sbac_pass_black',
             'sbac_pass_latinx',
             'sbac_pass_white',
             'sbac_pass_asian']]


In [67]:
percent_met.rename({'Seattle School District #1':'Seattle School District No. 1'},inplace=True)

## Qualifications

In [68]:
df1=df1[(df1.ContentAreaName=='All')&(df1.SchoolName.isnull())&(df1.SchoolYear=='2018-19')]

In [69]:
qualified=df1[df1.TeacherQualification\
                            .isin(['Inexperienced status',
                                   'Out-of-Field status',
                                   'Limited Certificated status'])][['DistrictName',
                                                                     'TeacherQualification',
                                                                     'TeacherPercent']]

In [70]:
qualified2=qualified.pivot(index='DistrictName',columns='TeacherQualification').droplevel(0,axis=1)

In [71]:
qualified2.columns=['teacher_percent_inexperienced',
                    'teacher_percent_limited_cert',
                    'teacher_percent_out_of_field']

## Enrollment

In [72]:
demographics=\
df3[(df3.SchoolName=='District Total')&\
    (df3.GradeLevel=='AllGrades')][['DistrictName',
                                    'All Students',
                                    'Black/ African American', 
                                    'Hispanic/ Latino of any race(s)', 
                                    'White', 
                                    'Asian']].set_index('DistrictName')

In [73]:
demographics['percent_black']=demographics['Black/ African American']/demographics['All Students']
demographics['percent_latinx']=demographics['Hispanic/ Latino of any race(s)']/demographics['All Students']
demographics['percent_white']=demographics['White']/demographics['All Students']
demographics['percent_asian']=demographics['Asian']/demographics['All Students']

In [74]:
demographics=demographics.rename({'All Students':'student_count'},axis=1)

In [75]:
demographics=demographics[['student_count',
                           'percent_black',
                           'percent_latinx',
                           'percent_white',
                           'percent_asian']]

## Teacher Demographics

In [76]:
teacher_demographics=\
df4[df4.SchoolName.isnull()&\
    df4.DemographicCategory.isin(['All',
                                  'Not Provided',
                                  'Asian',
                                  'White',
                                  'Black/African American',
                                  'Hispanic/Latino of any race(s)'])]\
                           .groupby(['DistrictName','DemographicCategory'])\
                           .sum()[['TeacherCount','TeacherPercent']]\
                           .reset_index()


In [77]:
teacher_demographics2=teacher_demographics.pivot(index='DistrictName',columns='DemographicCategory')

In [78]:
teacher_demographics3=teacher_demographics2.TeacherPercent.loc[:,'Asian':'White']

In [79]:
teacher_demographics3['num_teachers']=teacher_demographics2['TeacherCount','All']

In [80]:
teacher_demographics3.columns=['teacher_percent_asian', 
                               'teacher_percent_black', 
                               'teacher_percent_latinx',
                               'teacher_percent_unknown_demographic', 
                               'teacher_percent_white', 
                               'num_teachers']

In [81]:
teacher_demographics3=teacher_demographics3.fillna(0)[['num_teachers',
                                                       'teacher_percent_black',
                                                       'teacher_percent_latinx',
                                                       'teacher_percent_white',
                                                       'teacher_percent_asian', 
                                                       'teacher_percent_unknown_demographic',
                                                      ]]

# Funding

In [82]:
expenditure=\
df5.query("SchoolName=='District Total'")\
   .groupby('DistrictName').sum()

In [83]:
expenditure['student_FTE']=df5.query("SchoolName=='District Total'")\
                                  .groupby('DistrictName')\
                                  .mean().EnrollmentTotal


In [84]:
expenditure=expenditure[['student_FTE','Expenditure']].copy()

In [85]:
expenditure['spending_per_student']=expenditure.Expenditure/expenditure.student_FTE

In [86]:
expenditure.columns=['student_FTE','total_spending','spending_per_student']

# Merging Full List

In [87]:
df_wa=expenditure.join(teacher_demographics3,how='left')\
                 .join(qualified2,how='left')\
                 .join(demographics,how='left')\
                 .join(percent_met,how='outer')

In [88]:
df_wa.drop('Impact Public Schools',inplace=True)

In [93]:
df_wa.dropna(subset=['teacher_percent_black',
                     'teacher_percent_inexperienced'],inplace=True)

In [42]:
df_wa.to_csv('wa_school_district_data.csv')

# State Totals Dataset

In [43]:
state_totals=\
df2.query("GradeLevel=='10th Grade' and\
           TestAdministration=='SBAC' and\
           TestSubject=='Math' and\
           StudentGroup in ('All Students',\
                            'Asian','Black/ African American',\
                            'Hispanic/ Latino of any race(s)','White') and\
           SchoolName=='State Total'")[['StudentGroup', 
                                        'Count of students expected to test including previously passed',
                                        'CountMetStandard']\
                                      + ['PercentLevel'+str(i) for i in range(1,5)]]

In [44]:
state_totals.columns=['demographic',
                      'count_total_tested',
                      'count_met_standard',
                      'percent_level_1',
                      'percent_level_2',
                      'percent_level_3',
                      'percent_level_4']

In [45]:
state_totals=state_totals.reset_index(drop=True)

In [46]:
state_totals.percent_level_1=state_totals.percent_level_1.str.strip('%').astype(float)/100
state_totals.percent_level_2=state_totals.percent_level_2.str.strip('%').astype(float)/100
state_totals.percent_level_3=state_totals.percent_level_3.str.strip('%').astype(float)/100
state_totals.percent_level_4=state_totals.percent_level_4.str.strip('%').astype(float)/100

In [47]:
state_total=\
state_totals.astype({'count_total_tested':int,
                     'count_met_standard':int})

In [48]:
state_total.to_csv('state_totals.csv',index=False)

In [94]:
df_wa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307 entries, Aberdeen School District to Zillah School District
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   student_FTE                          307 non-null    float64
 1   total_spending                       307 non-null    float64
 2   spending_per_student                 307 non-null    float64
 3   num_teachers                         307 non-null    float64
 4   teacher_percent_black                307 non-null    float64
 5   teacher_percent_latinx               307 non-null    float64
 6   teacher_percent_white                307 non-null    float64
 7   teacher_percent_asian                307 non-null    float64
 8   teacher_percent_unknown_demographic  307 non-null    float64
 9   teacher_percent_inexperienced        307 non-null    float64
 10  teacher_percent_limited_cert         307 non-null    float64


In [92]:
state_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   demographic         5 non-null      object 
 1   count_total_tested  5 non-null      int64  
 2   count_met_standard  5 non-null      int64  
 3   percent_level_1     5 non-null      float64
 4   percent_level_2     5 non-null      float64
 5   percent_level_3     5 non-null      float64
 6   percent_level_4     5 non-null      float64
dtypes: float64(4), int64(2), object(1)
memory usage: 408.0+ bytes
